In [1]:
!pip install dspy pyaxis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.2/239.2 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 34.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.7/261.7 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 142.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.3/272.3 kB 22.5 MB/s eta 0:00:00
  Created wheel for pyaxis: filename=pyaxis-0.4.1-py3-none-any.whl size=14734 

In [2]:
import requests
import json
from pyaxis import pyaxis
from bs4 import BeautifulSoup
import pandas as pd
import dspy
import asyncio
from google.colab import userdata

MAKSTAT_URL = "https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/search"
API_URL = "https://makstat.stat.gov.mk:443/PXWeb/api/v1/mk/MakStat/"

lm = dspy.LM('openai/gpt-4o-mini', api_key=userdata.get('OpenAI_API'))
dspy.configure(lm=lm)

# Through UI

In [ ]:
from IPython.display import display, HTML
import sys
import textwrap

def keyword_search(keyword: str) -> dict[str, list[str]]:
    """A function that performs keyword search (in Macedonian) to retrieve a list of relevant tables from the National Statistics Office.
    """
    session = requests.Session()

    # Step 1: initial GET
    r = session.get(MAKSTAT_URL, params={"searchquery": keyword})
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")

    # Step 2: collect hidden inputs
    data = {}
    for inp in soup.select("input[type=hidden]"):
        if inp.get("name"):
            data[inp["name"]] = inp.get("value", "")

    # Step 3: add your desired options
    data["ctl00$ContentPlaceHolderMain$pxSearch$txtSearch"] = keyword
    data["ctl00$ContentPlaceHolderMain$pxSearch$cmdSearch"] = "Пребарај"
    data["ctl00$ContentPlaceHolderMain$searchOptions"] = "select"   # "Пребарај само"
    data["ctl00$ContentPlaceHolderMain$chkTitle"] = "on"            # "Наслов на табелата"

    # Step 4: POST back
    r2 = session.post(MAKSTAT_URL, data=data)
    r2.raise_for_status()

    # wrap in soup
    soup2 = BeautifulSoup(r2.text, "html.parser")

    rows = soup2.select("tr")
    results = {"titles": [], "links": [], "published": []}

    for row in rows:
        a = row.select_one("td.searchCellTable a")
        pub = row.select_one("td.searchCellPublished")
        if a and pub:
            results["titles"].append(a.get_text(strip=True))
            results["links"].append("https://makstat.stat.gov.mk" + a["href"])
            results["published"].append(pub.get_text(strip=True))

    return results

def get_table(url: str) -> pd.DataFrame:
    """A function that retrieves the full contents of the table in markdown. The table might be long, so use with caution only when necessary.

    Make sure the URL you provide as an argument is exactly what you see in the keyword search results!
    """
    API_BASE_URL = "https://makstat.stat.gov.mk:443/PXWeb/api/v1/"
    API_suffix = url.replace("MakStat__", "").replace("__", "/").split("pxweb/")[1]
    TABLE_URL = API_BASE_URL + API_suffix
    query = {
      "query": [],
      "response": {
        "format": "px"
      }
    }
    response = requests.post(TABLE_URL, json=query)
    if response.status_code == 200:
        print("Request successful!")
    else:
        print(f"Request failed with status code: {response.status_code}")

    with open("result.px", 'wb') as f:
        f.write(response.content)

    #try:
    result = pyaxis.parse('result.px', encoding='utf-8')
    #except UnicodeDecodeError:
    #    print("UTF-8 decoding failed, trying ISO-8859-5")
    #    result = pyaxis.parse('result.px', encoding='ISO-8859-5')


    table_df = pd.DataFrame(result['DATA'])

    return table_df.to_markdown()

react_agent = dspy.ReAct(
    signature="question -> answer",
    tools=[keyword_search, get_table],
    max_iters=5
)

#result = react_agent(question="Колку табели можеш да најдеш поврзани со трговија?")

stream_listeners = [
    # dspy.ReAct has a built-in output field called "next_thought".
    dspy.streaming.StreamListener(signature_field_name="next_thought", allow_reuse=True),
]
stream_react = dspy.streamify(react_agent, stream_listeners=stream_listeners)

async def read_output_stream(width=80):
    output = stream_react(question="Колку табели можеш да најдеш поврзани со трговија?")
    return_value = None
    async for chunk in output:
        if isinstance(chunk, dspy.streaming.StreamResponse):
            #print(chunk.chunk)
            raw = getattr(chunk, "chunk", chunk)
            if isinstance(raw, bytes):
                raw = raw.decode("utf-8", "replace")
            # remove leading newlines (common when server sends "\n" at chunk start)
            raw = raw.lstrip("\n")
            # if you also want to strip internal newlines, use replace:
            # raw = raw.replace("\n", " ")
            sys.stdout.write(raw)
            sys.stdout.flush()
        elif isinstance(chunk, dspy.Prediction):
            return_value = chunk
    return return_value

print(await read_output_stream())

Request successful!

Multilingual PX file


Request successful!

ERROR:pyaxis.pyaxis:Generic exception: Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pyaxis/pyaxis.py", line 142, in parse
    pc_axis = read(uri, encoding, timeout)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pyaxis/pyaxis.py", line 112, in read
    raw_pcaxis = file_object.read()
                 ^^^^^^^^^^^^^^^^^^
  File "<frozen codecs>", line 322, in decode
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb2 in position 220: invalid start byte



Имам проблем со добивањето на деталите за втората табела. Можеби ќе треба да направам ново пребарување за табели поврзани со трговија за да видиме дали можам да најдам друга табела или да добијам информации од друга извор.Не можам да добијам деталите за втората табела поради проблем со кодирањето. Можеби ќе треба да завршам со задачата, бидејќи веќе имам информации за првата табела и не можам да добијам дополнителни податоци.

Prediction(
    trajectory={'thought_0': 'Треба да направам пребарување за табели поврзани со трговија за да ги добијам 
потребните информации.', 'tool_name_0': 'keyword_search', 'tool_args_0': {'keyword': 'трговија'}, 'observation_0': 
{'titles': ['Е-трговија кај деловните субјекти од нефинансискиот сектор, со 10 или повеќе вработени (%)', 'Промет 
во трговијата со моторни возила, трговија на големо и на мало, по НКД Рев. 2, по месеци'], 'links': 
['https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__InfOpstestvo__DelovniSubjekti/325_InfOpst_Mk_12Enteco
m_ml.px/', 
'https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__VnatresnaTrgovija__VTBazna2021/125_Vt_mk_11_TrgDej_202
1_ml.px/'], 'published': ['08.4.2025', '01.9.2025']}, 'thought_1': 'Најдов две табели поврзани со трговија. Сега 
можам да ги добијам деталите за секоја од нив.', 'tool_name_1': 'get_table', 'tool_args_1': {'url': 
'https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__InfOpstestvo__DelovniSubjekti/325_InfOpst_Mk_12Entecom
_ml.px/'}, 'observation_1': '|    | Деловни субјекти (10+) кои имале е-трговија                                    
|   Година | DATA   
|\n|---:|:---------------------------------------------------------------------------------------------------------
-|---------:|:-------|\n|  0 | Деловни субјекти кои нарачале преку интернет или други компјутерски посредувани 
мрежи (е-купување)        |     2006 | 14.6   |\n|  1 | Деловни субјекти кои нарачале преку интернет или други 
компјутерски посредувани мрежи (е-купување)        |     2007 | 7.9    |\n|  2 | Деловни субјекти кои нарачале 
преку интернет или други компјутерски посредувани мрежи (е-купување)        |     2008 | 9.8    |\n|  3 | Деловни 
субјекти кои нарачале преку интернет или други компјутерски посредувани мрежи (е-купување)        |     2009 | 7.1 
|\n|  4 | Деловни субјекти кои нарачале преку интернет или други компјутерски посредувани мрежи (е-купување)       
|     2010 | 6.8    |\n|  5 | Деловни субјекти кои нарачале преку интернет или други компјутерски посредувани мрежи
(е-купување)        |     2011 | 8.1    |\n|  6 | Деловни субјекти кои нарачале преку интернет или други 
компјутерски посредувани мрежи (е-купување)        |     2012 | 5.1    |\n|  7 | Деловни субјекти кои нарачале 
преку интернет или други компјутерски посредувани мрежи (е-купување)        |     2013 | 4.6    |\n|  8 | Деловни 
субјекти кои нарачале преку интернет или други компјутерски посредувани мрежи (е-купување)        |     2014 | 5.0 
|\n|  9 | Деловни субјекти кои нарачале преку интернет или други компјутерски посредувани мрежи (е-купување)       
|     2015 | 5.9    |\n| 10 | Деловни субјекти кои нарачале преку интернет или други компјутерски посредувани мрежи
(е-купување)        |     2016 | 5.6    |\n| 11 | Деловни субјекти кои нарачале преку интернет или други 
компјутерски посредувани мрежи (е-купување)        |     2017 | 5.8    |\n| 12 | Деловни субјекти кои нарачале 
преку интернет или други компјутерски посредувани мрежи (е-купување)        |     2018 | "..."  |\n| 13 | Деловни 
субјекти кои нарачале преку интернет или други компјутерски посредувани мрежи (е-купување)        |     2019 | 
"..."  |\n| 14 | Деловни субјекти кои нарачале преку интернет или други компјутерски посредувани мрежи (е-купување)
|     2020 | "..."  |\n| 15 | Деловни субјекти кои нарачале преку интернет или други компјутерски посредувани мрежи
(е-купување)        |     2021 | "..."  |\n| 16 | Деловни субјекти кои нарачале преку интернет или други 
компјутерски посредувани мрежи (е-купување)        |     2022 | "..."  |\n| 17 | Деловни субјекти кои нарачале 
преку интернет или други компјутерски посредувани мрежи (е-купување)        |     2023 | "..."  |\n| 18 | Деловни 
субјекти кои нарачале преку интернет или други компјутерски посредувани мрежи (е-купување)        |     2024 | 
"..."  |\n| 19 | Деловни субјекти кои примиле нарачки преку интернет или други компјутерски посредувани мрежи 
(е-про

In [ ]:
result = react_agent(question="Пребарај табели поврзани со трговија и дај ми го линкот за табелата за моторни возила?")
print(result.answer)
print("Tool calls made:", result.trajectory)

Request successful!

Multilingual PX file


[Линк до табелата за регистрирани патни моторни и приклучни 
возила](https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__Transport__RegionalniStatistiki/125_Trans_RegVo
zila_ml.px/)

Tool calls made:
{
    'thought_0': 'Треба да извршам пребарување за табели поврзани со трговија, особено за моторни возила. Ова ќе ми
помогне да најдам линк до соодветната табела.',
    'tool_name_0': 'keyword_search',
    'tool_args_0': {'keyword': 'трговија моторни возила'},
    'observation_0': {
        'titles': [
            'Промет во трговијата со моторни возила, трговија на големо и на мало, по НКД Рев. 2, по месеци',
            'Просечна старост на патните моторни возила, 2003-2024',
            'Регистрирани патни моторни и приклучни возила , по години, по региони',
            'Регистрирани патни моторни и приклучни возила според видот и годината на производство, состојба 
31.12.2024',
            'Е-трговија кај деловните субјекти од нефинансискиот сектор, со 10 или повеќе вработени (%)',
            'Моторен сообраќај по категории на возила и вид на сообраќај во 2005, 2010 и 2015 година',
            'Дистрибуција на моторниот сообраќај по категории на возила според пописите во 2005, 2010 и 2015 
година',
            'Регистрирани патни моторни и приклучни превозни средства, по општини, по години, број',
            'Регистрирани патни моторни и приклучни превозни средства, прва регистрација, по општини, по години, 
број',
            'Регистрирани патни моторни превозни средства по гориво, по општини, по години, број'
        ],
        'links': [
            'https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__VnatresnaTrgovija__VTBazna2021/125_Vt_mk_1
1_TrgDej_2021_ml.px/',
            'https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__Transport__RegistriraniVozila/225_Trans_av
erageage_vehicle_ml.px/',
            'https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__Transport__RegionalniStatistiki/125_Trans_
RegVozila_ml.px/',
            'https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__Transport__RegistriraniVozila/250_Trans_go
dina_proizv_ml.px/',
            'https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__InfOpstestvo__DelovniSubjekti/325_InfOpst_
Mk_12Entecom_ml.px/',
            'https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__Transport__PopisEpatenSoobrakaj/550_Trans_
T2_MK_Motsoobr_mk.px/',
            'https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__Transport__PopisEpatenSoobrakaj/575_Trans_
Mk_Т41_mk.px/',
            'https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__Transport__RegistriraniVozila/150_Trans_re
gi_veh_by_type_mun_ml.px/',
            'https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__Transport__RegistriraniVozila/200_Trans_pr
v_regi_voz_po_vid_OPST_ml.px/',
            'https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__Transport__RegistriraniVozila/275_Trans_Re
g_voz_Gorivo_ml.px/'
        ],
        'published': [
            '01.9.2025',
            '08.8.2025',
            '07.8.2025',
            '08.8.2025',
            '08.4.2025',
            '21.11.2017',
            '21.11.2017',
            '12.8.2025',
            '12.8.2025',
            '06.8.2025'
        ]
    },
    'thought_1': 'Сега имам список на табели поврзани со моторни возила. Треба да го најдам линкот за табелата која
е најрелевантна за моторни возила. Најсоодветната табела изгледа да е "Регистрирани патни моторни и приклучни 
возила , по години, по региони". Ќе ја повикам функцијата get_table за да добијам целосна содржина на табелата.',
    'tool_name_1': 'get_table',
    'tool_args_1': {
        'url': 
'https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__Transport__RegionalniStatistiki/125_Trans_RegVozila_ml
.px/'
    },
    'observation_1': '|      | Регион               |   Година | Вид на возило       |   DATA 
|\n|-----:|:---------------------|---------:|:--------------------|-------:|\n|    0 | Република Македонија |     
2007 | Вкупно              | 287222 |\n|    1 | Република Македонија |     2007 | мотоцикли           |   4437 |\n|
2 | Република Македонија |     2007 | патнички автомоб

In [ ]:
result = react_agent(question="Прво најди ги табелите поврзани со образование, и потоа извлечи ја најинтересната и покажи ми ја.")
print(result.answer)
print("Tool calls made:", result.trajectory)

Request successful!

ERROR:pyaxis.pyaxis:Generic exception: Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pyaxis/pyaxis.py", line 142, in parse
    pc_axis = read(uri, encoding, timeout)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pyaxis/pyaxis.py", line 112, in read
    raw_pcaxis = file_object.read()
                 ^^^^^^^^^^^^^^^^^^
  File "<frozen codecs>", line 322, in decode
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xce in position 222: invalid continuation byte



Request successful!

ERROR:pyaxis.pyaxis:Generic exception: Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pyaxis/pyaxis.py", line 142, in parse
    pc_axis = read(uri, encoding, timeout)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pyaxis/pyaxis.py", line 112, in read
    raw_pcaxis = file_object.read()
                 ^^^^^^^^^^^^^^^^^^
  File "<frozen codecs>", line 322, in decode
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc7 in position 239: invalid continuation byte



Request successful!

Multilingual PX file


| Возраст   | Година | Степен на образование   | Пол    | DATA   |
|-----------|--------|------------------------|--------|--------|
| Вкупно    | 2017   | Вкупно                  | Вкупно | 154646 |
| Вкупно    | 2017   | Вкупно                  | Мажи   | 75560  |
| Вкупно    | 2017   | Вкупно                  | Жени   | 79086  |
| Вкупно    | 2017   | Од I до IV одделение    | Вкупно | "-"    |
| Вкупно    | 2017   | Од V до VIII одделение  | Вкупно | 2333   |
| Вкупно    | 2017   | Средно образование      | Вкупно | 72061  |
| Вкупно    | 2017   | Више образование        | Вкупно | 744    |
| Вкупно    | 2017   | Високо образование      | Вкупно | 72685  |
| Вкупно    | 2017   | Магистратура и докторат | Вкупно | 6823   |
| Вкупно    | 2018   | Вкупно                  | Вкупно | 152134 |
| Вкупно    | 2018   | Од V до VIII одделение  | Вкупно | 4159   |
| Вкупно    | 2018   | Средно образование      | Вкупно | 72355  |
| Вкупно    | 2018   | Више образование        | Вкупно | 1135   |
| Вкупно    | 2018   | Високо образование      | Вкупно | 67690  |
| Вкупно    | 2018   | Магистратура и докторат | Вкупно | 6529   |
| Вкупно    | 2019   | Вкупно                  | Вкупно | 154575 |
| Вкупно    | 2019   | Од V до VIII одделение  | Вкупно | 2253   |
| Вкупно    | 2019   | Средно образование      | Вкупно | 76587  |
| Вкупно    | 2019   | Више образование        | Вкупно | 1805   |
| Вкупно    | 2019   | Високо образование      | Вкупно | 67430  |
| Вкупно    | 2019   | Магистратура и докторат | Вкупно | 6274   |
| Вкупно    | 2020   | Вкупно                  | Вкупно | 150375 |
| Вкупно    | 2020   | Од V до VIII одделение  | Вкупно | 2735   |
| Вкупно    | 2020   | Средно образование      | Вкупно | 74992  |
| Вкупно    | 2020   | Више образование        | Вкупно | 1612   |
| Вкупно    | 2020   | Високо образование      | Вкупно | 64082  |
| Вкупно    | 2020   | Магистратура и докторат | Вкупно | 6811   |
| Вкупно    | 2021   | Вкупно                  | Вкупно | 158078 |
| Вкупно    | 2021   | Од V до VIII одделение  | Вкупно | 3791   |
| Вкупно    | 2021   | Средно образование      | Вкупно | 76418  |
| Вкупно    | 2021   | Више образование        | Вкупно | 1818   |
| Вкупно    | 2021   | Високо образование      | Вкупно | 69617  |
| Вкупно    | 2021   | Магистратура и докторат | Вкупно | 6434   |
| Вкупно    | 2022   | Вкупно                  | Вкупно | 158364 |
| Вкупно    | 2022   | Од V до VIII одделение  | Вкупно | 3448   |
| Вкупно    | 2022   | Средно образование      | Вкупно | 77590  |
| Вкупно    | 2022   | Више образование        | Вкупно | 1557   |
| Вкупно    | 2022   | Високо образование      | Вкупно | 69621  |
| Вкупно    | 2022   | Магистратура и докторат | Вкупно | 6096   |
| Вкупно    | 2023   | Вкупно                  | Вкупно | 152703 |
| Вкупно    | 2023   | Од V до VIII одделение  | Вкупно | 2444   |
| Вкупно    | 2023   | Средно образование      | Вкупно | 72463  |
| Вкупно    | 2023   | Више образование        | Вкупно | 802    |
| Вкупно    | 2023   | Високо образование      | Вкупно | 69814  |
| Вкупно    | 2023   | Магистратура и докторат | Вкупно | 5636   |
| Вкупно    | 2024   | Вкупно                  | Вкупно | 151015 |
| Вкупно    | 2024   | Од V до VIII одделение  | Вкупно | 3167   |
| Вкупно    | 2024   | Средно образование      | Вкупно | 72631  |
| Вкупно    | 2024   | Више образование        | Вкупно | nan    |
| Вкупно    | 2024   | Високо образование      | Вкупно | 67582  |
| Вкупно    | 2024   | Магистратура и докторат | Вкупно | 7146   |

Tool calls made:
{
    'thought_0': 'Морам да започнам со пребарување на табелите поврзани со образование. Ова ќе ми помогне да ги 
пронајдам релевантните информации за следниот чекор.',
    'tool_name_0': 'keyword_search',
    'tool_args_0': {'keyword': 'образование'},
    'observation_0': {
        'titles': [
            'ЦОР 4. Квалитетно образование',
            'Показатели од половите статистики - Образование',
            'Работоспособно население кое посетува редовно образование, според степенот на образование и возраста, 
по години',
            'Работоспособно население кое посетува редовно образование, според пол и полето на образование, по 
години',
            'Работоспособно население кое посетува редовно образование, според степенот на образование и возраста, 
по години, архивирани податоци',
            'Работоспособно население кое посетува редовно образование, според пол и полето на образование, по 
години, архивирани податоци',
            'Нецелосна програма за формално образование според економски статус и причини',
            'Дали нивото на завршено образование одговара на претходното работно место',
            'Лица кои посетуваат образование и доживотно учење по занимање',
            'Пречки за учество во образование и доживотно учење, по пол',
            'Пречки за учество во образование и доживотно учење, по возраст',
            'Работоспособно население според економската активност, пол и учеството во формалното (редовно) 
образование, степенот на образование, по години',
            'Работоспособно население според економската активност, пол и учеството во формалното (редовно) 
образование, степенот на образование, по години, архивирани податоци',
            'Вкупно резидентно население на возраст од 15+ години според полето на образование во највисокиот 
степен на образование, пол и возраст, по општини, Попис, 2021',
            'Вкупно резидентно население на возраст од 5+ години во формалното (редовно) образование, според 
степенот на образование, пол и возраст, по општини, Попис, 2021',
            'Вкупно резидентно население на возраст од 15+ години во формалното (редовно) образование според  поле 
на образование, пол и возраст, по општини, Попис, 2021',
            'Стапки на запишување во основно образование, по пол, по учебна година',
            'Стапки на запишување во средно образование, по пол, по учебна година',
            'Вработени (број на лица) во истражувачко развојна дејност според степенот на образование и секторот на
извршување',
            'Лица со низок степен на образование, по возрасни групи, по години',
            'Статус на учество во програми за формално образование според економски статус',
            'Степен до кој нивото на завршено образование одговара на главното работно место',
            'Степен до кој завршеното образование одговара на барањата на тековната работа',
            'Степен до кој професијата од завршеното образование одговара на барањата на работното место',
            'Лица кои посетуваат образование и доживотно учење според возраста, урбан и рурален дел',
            'Лица кои посетуваат образование и доживотно учење според полот и школската подготовка',
            'Лица кои посетуваат образование и доживотно учење според возраста и школската подготовка',
            'Лица кои посетуваат образование и доживотно учење според економската активност, по возараст',
            'Лица кои посетуваат образование и доживотно учење според економската активност, урбан и рурален дел',
            'Лица кои посетуваат образование и доживотно учење по сектори на дејности',
            'Учество во неформално образование, според видот, полето и бројот на образовните активности',
            'Учество во неформално образование, според видот на образовната активност и субјектот кој ја 
спроведува',
            'Просечен број на часови поминати во образование и обука, по возраст и по пол',
            'Просечен број на часови поминати во образование и о

# A better way

In [3]:
resp = requests.get(API_URL)
print(resp.status_code)
response = resp.json()
print(resp.json())

200
[{'id': 'VnatresnaTrgovija', 'type': 'l', 'text': 'Внатрешна трговија'}, {'id': 'Gradeznistvo', 'type': 'l', 'text': 'Градежништво'}, {'id': 'DelovniTend', 'type': 'l', 'text': 'Деловни тенденции'}, {'id': 'EkSmetkiZem', 'type': 'l', 'text': 'Економски сметки во земјоделството'}, {'id': 'Energija', 'type': 'l', 'text': 'Енергија'}, {'id': 'ZivotnaSredina', 'type': 'l', 'text': 'Животна средина'}, {'id': 'ZivotenStandard', 'type': 'l', 'text': 'Животен стандард'}, {'id': 'ZdrastveniSmetki', 'type': 'l', 'text': 'Здравствени сметки'}, {'id': 'Zemjodelstvo', 'type': 'l', 'text': 'Земјоделство'}, {'id': 'Industrija', 'type': 'l', 'text': 'Индустрија'}, {'id': 'InfOpstestvo', 'type': 'l', 'text': 'Информатичко општество'}, {'id': 'Kultura', 'type': 'l', 'text': 'Култура'}, {'id': 'NadvoresnaTrgovija', 'type': 'l', 'text': 'Надворешна трговија'}, {'id': 'Naselenie', 'type': 'l', 'text': 'Население'}, {'id': 'ObrazovanieNauka', 'type': 'l', 'text': 'Oбразование и наука'}, {'id': 'OdrzlivR

In [4]:
categories = {
    "name": [],
    'path': []
}
for entry in response:
    categories['name'].append(entry['text'])
    categories['path'].append(entry['id'])

categories = pd.DataFrame.from_dict(categories)

In [5]:
SYSTEM_PROMPT_CATEGORIES = f"""
Следниве категории на податоци се достапни, заедно со нивните paths:

{categories.to_markdown()}
"""

In [6]:
SYSTEM_PROMPT_CATEGORIES

'\nСледниве категории на податоци се достапни, заедно со нивните paths:\n\n|    | name                                | path               |\n|---:|:------------------------------------|:-------------------|\n|  0 | Внатрешна трговија                  | VnatresnaTrgovija  |\n|  1 | Градежништво                        | Gradeznistvo       |\n|  2 | Деловни тенденции                   | DelovniTend        |\n|  3 | Економски сметки во земјоделството  | EkSmetkiZem        |\n|  4 | Енергија                            | Energija           |\n|  5 | Животна средина                     | ZivotnaSredina     |\n|  6 | Животен стандард                    | ZivotenStandard    |\n|  7 | Здравствени сметки                  | ZdrastveniSmetki   |\n|  8 | Земјоделство                        | Zemjodelstvo       |\n|  9 | Индустрија                          | Industrija         |\n| 10 | Информатичко општество              | InfOpstestvo       |\n| 11 | Култура                             | Kultura  

In [17]:
import os
import chardet

def go_down_path(path: str, base_path=API_URL):
    """Makes a new request with the path segment of choice with respect to the base path.
    """

    try:
        new_url = os.path.join(base_path, path)
        resp_list = requests.get(new_url).json()

        for entry in resp_list:
            if entry is None:
                continue
            entry["full_path"] = os.path.join(new_url, entry["id"])
            if entry["type"] == "t":
                entry["type"] = "table"
            else:
                entry["type"] = "list"
        return resp_list
    except Exception as e:
        print(f"Failed with error {e}")
        return None

def get_table(table_url: str, output_format: str = "markdown"):
    """Retrieves the full contents of the table.
    Params:
        table_url: the URL from which to retrieve the table
        output_format: the format in which to output the table; can be "markdown" or "dataframe"
    """
    query = {
      "query": [],
      "response": {
        "format": "px"
      }
    }

    try:
        response = requests.post(table_url, json=query)
    except:
        print("Invalid URL")

    if response.status_code == 200:

        print("Request successful!")

        with open("result.px", 'wb') as f:
            f.write(response.content)

        try:
            with open("result.px", 'rb') as f:
                raw = f.read()
            detected = chardet.detect(raw)
            result = pyaxis.parse('result.px', encoding=detected['encoding'])#'utf-8')

            table_df = pd.DataFrame(result['DATA'])

            if output_format == "markdown":
                return table_df.to_markdown()
            elif output_format == "dataframe":
                return table_df
            else:
                print("Invalid output format")
                return None

        except Exception as e:
            print(f"Error during parsing: {e}")
    else:
        print(f"Request failed with status code: {response.status_code}")
        return None


        #result = pyaxis.parse('result.px', encoding='cp1251')
        #result = pyaxis.parse('result.px', encoding='ISO-8859-5')


go_down_path("ProstorniEdinici")

[{'id': '125_PrEdn_reg_09_BrOpstiniNM_ml.px',
  'type': 'table',
  'text': 'Број на општини и населени места, по региони',
  'updated': '2023-06-02T12:00:00',
  'full_path': 'https://makstat.stat.gov.mk:443/PXWeb/api/v1/mk/MakStat/ProstorniEdinici/125_PrEdn_reg_09_BrOpstiniNM_ml.px'}]

In [8]:
go_down_path("ObrazovanieNauka")

[{'id': 'OsnovnoObrazovanie',
  'type': 'list',
  'text': 'Основно образование',
  'full_path': 'https://makstat.stat.gov.mk:443/PXWeb/api/v1/mk/MakStat/ObrazovanieNauka/OsnovnoObrazovanie'},
 {'id': 'SrednoObrazovanie',
  'type': 'list',
  'text': 'Средно образование',
  'full_path': 'https://makstat.stat.gov.mk:443/PXWeb/api/v1/mk/MakStat/ObrazovanieNauka/SrednoObrazovanie'},
 {'id': 'VisokoObrazovanie',
  'type': 'list',
  'text': 'Високо образование',
  'full_path': 'https://makstat.stat.gov.mk:443/PXWeb/api/v1/mk/MakStat/ObrazovanieNauka/VisokoObrazovanie'},
 {'id': 'Nauka',
  'type': 'list',
  'text': 'Наука',
  'full_path': 'https://makstat.stat.gov.mk:443/PXWeb/api/v1/mk/MakStat/ObrazovanieNauka/Nauka'},
 {'id': 'DozivotnoUcenje',
  'type': 'list',
  'text': 'Стручна обука и доживотно учење',
  'full_path': 'https://makstat.stat.gov.mk:443/PXWeb/api/v1/mk/MakStat/ObrazovanieNauka/DozivotnoUcenje'},
 {'id': 'PolindikatorObrazovanie',
  'type': 'list',
  'text': 'Индикатори по по

In [9]:
import dspy
from google.colab import userdata

lm = dspy.LM('openai/gpt-4.1-mini', api_key=userdata.get('OpenAI_API'))
dspy.configure(lm=lm)

class DSPy_makstat(dspy.Signature):
    f"""Ти си агент кој ја пребарува МАКСТАТ датабазата на Државниот Завод за Статистика во Македонија.

Корисникот ќе те праша прашање, и твојата задача е одиш до релевантната табела, почнувајќи од основните категории, и да ја извлечеш табелата.

Податоците од тип 'l' се листи, а тие од тип 't' се табели.

{SYSTEM_PROMPT_CATEGORIES}"""

    user_request: str = dspy.InputField()
    process_result: str = dspy.OutputField(
        desc=(
                "Порака која одговара на прашањето на корисникот."
            )
        )


agent = dspy.ReAct(
    signature=DSPy_makstat,
    tools=[go_down_path, get_table],
    max_iters=2
)

In [10]:
result = agent(user_request="дај ми ја табелата за плати во 2021")
print(result)

Failed with error Expecting value: line 1 column 1 (char 0)
Failed with error Expecting value: line 1 column 1 (char 0)
Prediction(
    trajectory={'thought_0': 'To find the table for salaries in 2021, I will first navigate to the relevant section in the MakStat database that likely contains salary data. I will start by exploring the main categories related to labor or earnings.', 'tool_name_0': 'go_down_path', 'tool_args_0': {'path': 'Trziste_na_rabota', 'base_path': 'https://makstat.stat.gov.mk:443/PXWeb/api/v1/mk/MakStat/'}, 'observation_0': None, 'thought_1': 'I have accessed the labor market section. Now I will look for a subsection related to salaries or earnings to find the table for salaries in 2021.', 'tool_name_1': 'go_down_path', 'tool_args_1': {'path': 'Plati', 'base_path': 'https://makstat.stat.gov.mk:443/PXWeb/api/v1/mk/MakStat/Trziste_na_rabota'}, 'observation_1': None},
    reasoning='I have navigated to the "Trziste_na_rabota" (Labor Market) section and then to the "Pl

In [11]:
dspy.inspect_history()





[2025-09-26T08:07:01.087398]

System message:

Your input fields are:
1. `user_request` (str): 
2. `trajectory` (str):
Your output fields are:
1. `reasoning` (str): 
2. `process_result` (str): Порака која одговара на прашањето на корисникот.
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## user_request ## ]]
{user_request}

[[ ## trajectory ## ]]
{trajectory}

[[ ## reasoning ## ]]
{reasoning}

[[ ## process_result ## ]]
{process_result}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `user_request`, produce the fields `process_result`.


User message:

[[ ## user_request ## ]]
дај ми ја табелата за плати во 2021

[[ ## trajectory ## ]]
[[ ## thought_0 ## ]]
To find the table for salaries in 2021, I will first navigate to the relevant section in the MakStat database that likely contains salary data. I will start by exploring the main categories related to labor or earnings.

[[

# Eval set

In [12]:
categories = {
    "name": [],
    'path': []
}
for entry in response:
    categories['name'].append(entry['text'])
    categories['path'].append(entry['id'])

categories = pd.DataFrame.from_dict(categories)

In [13]:
import random

random.choice(categories['path'])

'Zemjodelstvo'

In [14]:
categories['path']

,path
0,VnatresnaTrgovija
1,Gradeznistvo
2,DelovniTend
3,EkSmetkiZem
4,Energija
5,ZivotnaSredina
6,ZivotenStandard
7,ZdrastveniSmetki
8,Zemjodelstvo
9,Industrija


In [15]:
# generate a dataframe with dummy data

df = pd.DataFrame({
    'Name': ['John', 'Jane', 'Bob', 'Alice', 'Tom'],
    'Age': [25, 30, 35, 40, 45],
    'Salary': [50000, 60000, 70000, 80000, 90000],
    'Department': ['IT', 'HR', 'IT', 'Finance', 'HR']
})

In [26]:
import requests
import json
import os
import re
from google.colab import userdata

url = "https://openrouter.ai/api/v1/chat/completions"
headers = {
  "Authorization": f"Bearer {userdata.get('OPENROUTER_API_KEY')}",
  "Content-Type": "application/json"
}

def extract_json(raw_output):
    # Find all fenced code blocks
    candidates = re.findall(r"```(?:json)?\s*(.*?)\s*```", raw_output, flags=re.DOTALL | re.IGNORECASE)

    # Always include the whole raw string as a fallback candidate
    candidates.append(raw_output)

    for cand in candidates:
        try:
            parsed = json.loads(cand)
            return parsed, cand.strip()
        except json.JSONDecodeError:
            continue

    raise ValueError("No valid JSON found in output.")


def generate_synthetic_data(table, N=10):
    """
    Params:
    - tables: list of table (in pandas DataFrame format) to generate synthetic data for
    - N: number of synthetic data points to generate for each table
    """

    system_prompt = f"""

    Ти си професионален data engineer. Твојата задача е да генерираш прашања и клуч на точни SQL queries и одговори базирани врз основа на табела која ќе ти биде дадена.

    Прашањата треба да бидат генерални прашања кои би ги прашал некој граѓанин, или новинар, кој ја користи веб платформата на Државниот Завод за Статистика. Значи корисникот е заинтересиран за некаква статистика. КОРИСНИКОТ НЕМА ПРИСТАП ДО ТАБЕЛАТА. Прашањето кое го генерираш треба да биде генерално и самостојно. Сепак, важно е прашањата да бидат одговорливи врз основа на содржината во табелата!

    За секоја табела, треба да одговориш во JSON format со следниве полиња:

    - question: прашањето, на македонски јазик, во врска со податоците од табелата
    - sql_query: SQL query кое е потребно за извлекување на одговорот, или во форма на број или во форма на под-табела
    - answer: точниот одговор на прашањето; ако е табела, тогаш ова поле треба да биде "<SQL>"

    За секоја табела, генерирај {N} примери од question, sql_query и answer.
    """

    descriptive_input = f"""
    Here is an overview of the table you will be working with.

    Shape of the dataframe:
    {table.shape}

    Column data types:
    {table.dtypes.to_frame("dtype").to_markdown()}

    Fraction of missing values per column:
    {table.isnull().mean().to_frame("null_fraction").to_markdown()}

    Basic descriptive statistics:
    {table.describe(include="all").to_markdown()}

    Unique value samples (up to 10) for categorical/object columns:
    {
        {col: table[col].dropna().unique()[:10].tolist()
        for col in table.select_dtypes(include=["object", "category"]).columns}
    }

    Minimum and maximum for datetime columns:
    {
        {col: {"min": str(table[col].min()), "max": str(table[col].max())}
        for col in df.select_dtypes(include=["datetime64[ns]"]).columns}
    }

    Top 10 most frequent values for categorical columns:
    {
        {col: table[col].value_counts().head(10).to_dict()
        for col in table.select_dtypes(include=["object", "category"]).columns}
    }

    Here are the first 20 rows of the table:
    {table.head(20).to_markdown()}
    """


    payload = {
    "model": "openai/gpt-4.1-mini",
    "messages": [
        {
        "role": "system",
        "content": system_prompt
        },
        {
        "role": "user",
        "content": descriptive_input
        }
    ]
    }

    response = requests.post(url, headers=headers, json=payload)

    try:
        raw_output = response.json()['choices'][0]['message']['content']

        # clean and parse in json
        #cleaned = re.sub(r"^```json|```$", "", raw_output, flags=re.MULTILINE).strip()
        #return json.loads(cleaned)

        parsed, cleaned = extract_json(raw_output)

        return parsed

    except Exception as e:
        print(f"Error: {e}; Response: {response.json()}")
        return None


eval_row = generate_synthetic_data(df)

eval_row

[{'question': 'Кој е просечниот доход на вработените во компанијата?',
  'sql_query': 'SELECT AVG(Salary) AS average_salary FROM table;',
  'answer': 70000},
 {'question': 'Кои се имињата на сите вработени и нивната старост?',
  'sql_query': 'SELECT Name, Age FROM table;',
  'answer': '<SQL>'},
 {'question': 'Која е најголемата плата која некој вработен ја прима во компанијата?',
  'sql_query': 'SELECT MAX(Salary) AS max_salary FROM table;',
  'answer': 90000},
 {'question': 'Колку вработени има во одделот за IT?',
  'sql_query': "SELECT COUNT(*) AS count_it FROM table WHERE Department = 'IT';",
  'answer': 2},
 {'question': 'Која е најмладата возраст меѓу вработените?',
  'sql_query': 'SELECT MIN(Age) AS youngest_age FROM table;',
  'answer': 25},
 {'question': 'Колку вработени има во секој оддел поединечно?',
  'sql_query': 'SELECT Department, COUNT(*) AS number_of_employees FROM table GROUP BY Department;',
  'answer': '<SQL>'},
 {'question': 'Која е средната возраст на вработените 

In [27]:
print(eval_row)

[{'question': 'Кој е просечниот доход на вработените во компанијата?', 'sql_query': 'SELECT AVG(Salary) AS average_salary FROM table;', 'answer': 70000}, {'question': 'Кои се имињата на сите вработени и нивната старост?', 'sql_query': 'SELECT Name, Age FROM table;', 'answer': '<SQL>'}, {'question': 'Која е најголемата плата која некој вработен ја прима во компанијата?', 'sql_query': 'SELECT MAX(Salary) AS max_salary FROM table;', 'answer': 90000}, {'question': 'Колку вработени има во одделот за IT?', 'sql_query': "SELECT COUNT(*) AS count_it FROM table WHERE Department = 'IT';", 'answer': 2}, {'question': 'Која е најмладата возраст меѓу вработените?', 'sql_query': 'SELECT MIN(Age) AS youngest_age FROM table;', 'answer': 25}, {'question': 'Колку вработени има во секој оддел поединечно?', 'sql_query': 'SELECT Department, COUNT(*) AS number_of_employees FROM table GROUP BY Department;', 'answer': '<SQL>'}, {'question': 'Која е средната возраст на вработените во одделот HR?', 'sql_query': 

In [ ]:
import time

eval_dataset = []
for i in range(100):

    # choose a random category and get the options
    print(f"ITERATION {i}")
    cat = random.choice(categories['path'])
    print("└──", cat)
    cat_structure = go_down_path(cat)

    # if there are just tables, get em
    if cat_structure[0]['type'] == 'table':
        tab = random.choice(cat_structure)['full_path']
        print(get_table(tab))

    # if the subcategories are lists, recursively go down the tree
    elif cat_structure[0]['type'] == 'list':
        indent = 1
        while cat_structure[0]['type'] == 'list':
            time.sleep(4)
            cat = random.choice(cat_structure)
            to_print = "    " * indent + "└──" + cat['text']
            print(to_print)

            cat_structure = go_down_path(cat['id'], base_path=os.path.dirname(cat['full_path']))

            if cat_structure is None:
                print("Empty, passing this one")
                break
            else:
                if cat_structure[0]['type'] == 'table':
                    tab = random.choice(cat_structure)['full_path']
                    print("Retrieving table from path ", tab)
                    table_df = get_table(tab, output_format="dataframe")
                    for _ in range(10):
                        if table_df is not None:
                            data_row = generate_synthetic_data(table_df)
                            if data_row is not None:
                                data_row['table_url'] = tab
                                eval_dataset.extend(data_row)
                    break
            indent += 1


from datasets import Dataset

# massage it in the form required by hf
keys = list(eval_dataset[0].keys())
dataset_dict = {
    key: [str(entry[key]) for entry in eval_dataset] for key in keys
}
# convert and upload
ds = Dataset.from_dict(dataset_dict)
from huggingface_hub import login
login(token=userdata.get("huggingface_token"))
ds.push_to_hub("ilijalichkovski/makstat_QA")

ITERATION 0
└── BDP
    └──Бруто-домашен производ тримесечни податоци
        └──Архива - Тримесечни БДП според ЕСС 95
Retrieving table from path  https://makstat.stat.gov.mk:443/PXWeb/api/v1/mk/MakStat/BDP/BDPTrimesecni/TrimesecniBDPsporedESS95/125_NacSmA_Mk_01ProKv_mk.px
Request successful!
ITERATION 1
└── Sudstvo
    └──Архива
Retrieving table from path  https://makstat.stat.gov.mk:443/PXWeb/api/v1/mk/MakStat/Sudstvo/SudstvoArhiva/102_SK1_Mk_T3a_ml.px
Request successful!
Multilingual PX file
ITERATION 2
└── EkSmetkiZem
Request successful!
Multilingual PX file
|      | Земјоделско производство и трошоци на земјоделското производство          |   Година | DATA                  |
|-----:|:--------------------------------------------------------------------------|---------:|:----------------------|
|    0 | 01 Жита                                                                   |     1998 | 6306.00000000000000   |
|    1 | 01 Жита                                                       

KeyboardInterrupt: 

# Optimization with DSPy

In [28]:
test_table_url = "https://makstat.stat.gov.mk:443/PXWeb/api/v1/mk/MakStat/DelovniSubj/AktDelovniSubjekti/400_Delovni_Reg_Ops_Golemina_Vraboteni_ml.px"


test_table = get_table(test_table_url, output_format="dataframe")

generate_synthetic_data(test_table)

Request successful!
Multilingual PX file


[{'question': 'Колкав е вкупниот број на активни претпријатија во целата земја за секоја година?',
  'sql_query': "SELECT Година, DATA AS Вкупно_претпријатија FROM табела WHERE Регион/Општина = 'Вкупно' AND Број на активни претпријатија според бројот на вработени = 'Вкупно' ORDER BY Година;",
  'answer': '<SQL>'},
 {'question': 'Кој регион има најмногу активни претпријатија со вработени од 1 до 9 во 2023 година?',
  'sql_query': "SELECT Регион/Општина, DATA FROM табела WHERE Година = '2023' AND Број на активни претпријатија според бројот на вработени = '1-9' AND Регион/Општина != 'Вкупно' ORDER BY CAST(DATA AS INTEGER) DESC LIMIT 1;",
  'answer': '<SQL>'},
 {'question': 'Што е вкупниот број на активни претпријатија без вработени (0 вработени) во Вардарски регион во 2022 година?',
  'sql_query': "SELECT DATA FROM табела WHERE Регион/Општина = 'Вардарски регион' AND Година = '2022' AND Број на активни претпријатија според бројот на вработени = '0';",
  'answer': '<SQL>'},
 {'question': '

In [29]:
eval_set = []
for i in range(10):
    eval_set.append(generate_synthetic_data(test_table))

In [30]:
eval_set

[[{'question': 'Колкав е вкупниот број на активни претпријатија во Република Северна Македонија за секоја година во периодот 2021-2023?',
   'sql_query': "SELECT Година, DATA AS Вкупен_број_на_активни_претпријатија FROM tablename WHERE Регион_Општина = 'Вкупно' AND Број_на_активни_претпријатија_според_бројот_на_вработени = 'Вкупно' ORDER BY Година;",
   'answer': '<SQL>'},
  {'question': 'Кој регион има најголем број на активни претпријатија со 1-9 вработени во 2023 година?',
   'sql_query': "SELECT Регион_Општина, DATA FROM tablename WHERE Година = '2023' AND Број_на_активни_претпријатија_според_бројот_на_вработени = '1-9' ORDER BY CAST(DATA AS INTEGER) DESC LIMIT 1;",
   'answer': '<SQL>'},
  {'question': 'Каков е трендот на бројот на претпријатија со над 250 вработени во целата држава од 2021 до 2023?',
   'sql_query': "SELECT Година, DATA FROM tablename WHERE Регион_Општина = 'Вкупно' AND Број_на_активни_претпријатија_според_бројот_на_вработени = '250 +' ORDER BY Година;",
   'answ

In [54]:
len(eval_set)

10

In [56]:
joined = [x for sublist in eval_set for x in sublist]
len(joined)

100

In [65]:
def get_table_context(table):
    descriptive_input = f"""Еве брз преглед на табелата која ти е на располагање.

    Shape of the dataframe:
    {table.shape}

    Податочни типови во колоните:
    {table.dtypes.to_frame("dtype").to_markdown()}

    Пропорција од вредностите кои недостасуваат во секоја колона:
    {table.isnull().mean().to_frame("null_fraction").to_markdown()}

    Базични карактеристики:
    {table.describe(include="all").to_markdown()}

    Примери од уникатни вредности (максимум 10) во колоните кои содржат објекти или категории:
    {
        {col: table[col].dropna().unique()[:10].tolist()
        for col in table.select_dtypes(include=["object", "category"]).columns}
    }

    Минимум и максимум за колоните со датуми:
    {
        {col: {"min": str(table[col].min()), "max": str(table[col].max())}
        for col in df.select_dtypes(include=["datetime64[ns]"]).columns}
    }

    Топ 10 најчести вредности во колоните кои содржат категории:
    {
        {col: table[col].value_counts().head(10).to_dict()
        for col in table.select_dtypes(include=["object", "category"]).columns}
    }

    Првите 20 реда од табелата:
    {table.head(20).to_markdown()}
    """

    return descriptive_input

context_field = get_table_context(test_table)

for sample in joined:
    sample['table_context'] = context_field

In [66]:
joined

[{'question': 'Колкав е вкупниот број на активни претпријатија во Република Северна Македонија за секоја година во периодот 2021-2023?',
  'sql_query': "SELECT Година, DATA AS Вкупен_број_на_активни_претпријатија FROM tablename WHERE Регион_Општина = 'Вкупно' AND Број_на_активни_претпријатија_според_бројот_на_вработени = 'Вкупно' ORDER BY Година;",
  'answer': '<SQL>',
  'table_context': 'Еве брз преглед на табелата која ти е на располагање. \n\n    Shape of the dataframe:\n    (1869, 4)\n\n    Податочни типови во колоните:\n    |                                                          | dtype   |\n|:---------------------------------------------------------|:--------|\n| Регион/Општина                                           | object  |\n| Година                                                   | object  |\n| Број на активни претпријатија според бројот на вработени | object  |\n| DATA                                                     | object  |\n\n    Пропорција од вредностите к

In [74]:
import dspy
from datasets import load_dataset

def init_dataset():
    dspy_set = [
        dspy.Example({
            "question": x['question'],
            "table_context": x['table_context'],
            'sql_query': x['sql_query']
        }).with_inputs("question", "table_context")
        for x in joined
    ]
    import random
    random.Random(0).shuffle(dspy_set)
    tot_num = len(dspy_set)

    train_set = dspy_set[:int(0.7 * tot_num)]
    val_set = dspy_set[int(0.7 * tot_num):int(0.85 * tot_num)]
    test_set = dspy_set[int(0.85 * tot_num):] # repeat 5 times for statistical stability

    return train_set, val_set, test_set

In [75]:
train_set, val_set, test_set = init_dataset()

print(len(train_set), len(val_set), len(test_set))

70 15 15


In [76]:
print("Question:")
print(train_set[0]['question'])
print("SQL query:")
print(train_set[0]['sql_query'])
print("Table context:")
print(train_set[0]['table_context'])

Question:
Каков е бројот на активни претпријатија со 0 вработени во Велес за секоја година?
SQL query:
SELECT Година, DATA FROM table WHERE Регион/Општина = 'Велес' AND "Број на активни претпријатија според бројот на вработени" = '0' ORDER BY Година;
Table context:
Еве брз преглед на табелата која ти е на располагање. 

    Shape of the dataframe:
    (1869, 4)

    Податочни типови во колоните:
    |                                                          | dtype   |
|:---------------------------------------------------------|:--------|
| Регион/Општина                                           | object  |
| Година                                                   | object  |
| Број на активни претпријатија според бројот на вработени | object  |
| DATA                                                     | object  |

    Пропорција од вредностите кои недостасуваат во секоја колона:
    |                                                          |   null_fraction |
|:-------------------

In [81]:
class GenerateSQL(dspy.Signature):
    """Provide the right SQL given a table and a question."""
    table_context: str = dspy.InputField(desc="the details of the table")
    question: str = dspy.InputField()
    answer: str = dspy.OutputField(desc="the SQL query to answer the question")

program = dspy.ChainOfThought(GenerateSQL)

program(table_context=get_table_context(test_table), question="Каков е бројот на активни претпријатија со 0 вработени во Велес за секоја година?")

Prediction(
    reasoning='Прашувањето бара да се извлече бројот на активни претпријатија со 0 вработени во општина Велес за секоја година. Во табелата, колоната "Регион/Општина" содржи името на општината, колоната "Година" ја содржи годината, колоната "Број на активни претпријатија според бројот на вработени" ја содржи категоријата на бројот на вработени, а колоната "DATA" го содржи бројот на претпријатија. Треба да филтрираме по "Регион/Општина" = \'Велес\' и "Број на активни претпријатија според бројот на вработени" = \'0\', и да ги прикажеме резултатите по година.',
    answer='SELECT Година, DATA\nFROM table_name\nWHERE "Регион/Општина" = \'Велес\'\n  AND "Број на активни претпријатија според бројот на вработени" = \'0\'\nORDER BY Година;'
)

In [ ]:
def get_variable_name(df):
    for name, obj in globals().items():
        if obj is df:
            return name
    return None

def parse_query(query):
    if "```sql" in query:
        return query.replace("```sql", "").replace("```", "").strip()
    elif "```" in query:
        return query.replace("```", "").strip()
    else:
        return None

parsed_query = parse_query(sql_query)
print(f"parsed_query: {parsed_query}")

run_query = lambda q: sqldf(q, globals())

result_df = run_query(parsed_query)

display(result_df)

In [96]:
!pip install pandasql

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26773 sha256=ec9083dfb723cd04d7e0fd0aaf8f2cb63992907cea7ac4cc3d4f07467a51c298
  Stored in directory: /root/.cache/pip/wheels/15/a1/e7/6f92f295b5272ae5c02365e6b8fa19cb93f16a537090a1cf27
Successfully built pandasql


In [107]:
from pandasql import sqldf
run_query = lambda q: sqldf(q, globals())

def metric(example, prediction, trace=None, pred_name=None, pred_trace=None):
    correct_answer = (example['sql_query'])
    correct_df = run_query(correct_answer)
    try:
        llm_answer = (prediction.answer)
        predicted_df = run_query(llm_answer)
        display(predicted_df)
    except ValueError as e:
        return 0
    return int(correct_answer == llm_answer)

In [108]:
table_name = test_table # quick fix for now -- this needs to be provided to the synthetic data generation model in context so it uses that table name

metric(test_set[3], program(table_context=get_table_context(test_table), question=test_set[3]['question']))

PandaSQLException: (sqlite3.OperationalError) near "table": syntax error
[SQL: SELECT `Регион/Општина`, DATA FROM table WHERE Година='2021' AND `Број на активни претпријатија според бројот на вработени`='Вкупно' ORDER BY CAST(DATA AS INTEGER) DESC LIMIT 1;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [90]:
import dspy
evaluate = dspy.Evaluate(
    devset=test_set,
    metric=metric,
    num_threads=1,
    display_table=True,
    display_progress=True
)

evaluate(program)

Average Metric: 0.00 / 15 (0.0%): 100%|██████████| 15/15 [00:26<00:00,  1.75s/it]

2025/09/26 10:12:23 INFO dspy.evaluate.evaluate: Average Metric: 0 / 15 (0.0%)


,question,table_context,sql_query,reasoning,answer,metric
0,Каков е вкупниот број активни претпријатија со 20-49 вработени за ...,Еве брз преглед на табелата која ти е на располагање. Shape of the...,SELECT SUM(CAST(DATA AS INTEGER)) FROM table WHERE Година = '2022'...,Прашувањето бара да се најде вкупниот број на активни претпријатиј...,"SELECT DATA\nFROM table_name\nWHERE ""Регион/Општина"" = 'Вкупно'\n ...",
1,Какви се трендовите на бројот на активни претпријатија со 50-249 в...,Еве брз преглед на табелата која ти е на располагање. Shape of the...,"SELECT Година, DATA FROM table WHERE `Регион/Општина` = 'Вкупно' A...",За да ги анализираме трендовите на бројот на активни претпријатија...,"SELECT Година, CAST(DATA AS INTEGER) AS Број_на_активни_претпријат...",
2,Каква е дистрибуцијата на активни претпријатија по бројот на врабо...,Еве брз преглед на табелата која ти е на располагање. Shape of the...,"SELECT ""Број на активни претпријатија според бројот на вработени"",...",Прашувањето бара да се прикаже дистрибуцијата на активни претприја...,"SELECT \n ""Број на активни претпријатија според бројот на врабо...",
3,Кој регион имаше најголем број на активни претпријатија во 2021 го...,Еве брз преглед на табелата која ти е на располагање. Shape of the...,"SELECT `Регион/Општина`, DATA FROM table WHERE Година='2021' AND `...",Прашувањето бара да се најде регион со најголем број на активни пр...,"SELECT ""Регион/Општина"", SUM(CAST(""DATA"" AS INTEGER)) AS total_act...",
4,Колку бил бројот на активни претпријатија со вработени од 10 до 19...,Еве брз преглед на табелата која ти е на располагање. Shape of the...,SELECT DATA FROM table WHERE Регион/Општина = 'Велес' AND Година =...,Прашувањето бара да се најде бројот на активни претпријатија со бр...,"SELECT DATA\nFROM table_name\nWHERE ""Регион/Општина"" = 'Велес'\n ...",
5,Кои региони имаат најголем број на активни претпријатија со 1-9 вр...,Еве брз преглед на табелата која ти е на располагање. Shape of the...,"SELECT `Регион/Општина`, DATA FROM table WHERE Година = '2022' AND...",За да ги најдеме регионите со најголем број на активни претпријати...,"SELECT ""Регион/Општина"", ""DATA"" FROM table_name WHERE ""Година"" = '...",
6,Каква е вкупната бројка на активни претпријатија во општина Неготи...,Еве брз преглед на табелата која ти е на располагање. Shape of the...,SELECT DATA FROM table WHERE Регион/Општина = 'Неготино' AND Годин...,За да ја најдеме вкупната бројка на активни претпријатија во општи...,"SELECT DATA\nFROM table_name\nWHERE ""Регион/Општина"" = 'Неготино'\...",
7,Колку активни претпријатија со 1-9 вработени имаше во Кумановскиот...,Еве брз преглед на табелата која ти е на располагање. Shape of the...,SELECT DATA FROM table WHERE Регион/Општина='Кумановски регион' AN...,"За да одговориме на прашањето, треба да ги филтрираме податоците з...","SELECT DATA\nFROM table_name\nWHERE ""Регион/Општина"" = 'Кумановски...",
8,Колкав е бројот на активни претпријатија без вработени (0 вработен...,Еве брз преглед на табелата која ти е на располагање. Shape of the...,SELECT DATA FROM table WHERE `Регион/Општина` = 'Вардарски регион'...,Прашувањето бара да се најде бројот на активни претпријатија со 0 ...,"SELECT DATA\nFROM table_name\nWHERE ""Регион/Општина"" = 'Вардарски ...",
9,Која година има најмногу активни претпријатија со 250+ вработени в...,Еве брз преглед на табелата која ти е на располагање. Shape of the...,"SELECT Година, DATA FROM table WHERE `Регион/Општина` = 'Вкупно' A...",За да ја најдеме годината со најмногу активни претпријатија со 250...,"SELECT Година FROM table_name WHERE ""Регион/Општина"" = 'Вкупно' AN...",


EvaluationResult(score=0.0, results=<list of 15 results>)